In [1]:
pip install numpy-stl

In [2]:
!pip install vtk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 MB 10.9 MB/s eta 0:00:00


In [3]:
!pip install networkx

In [4]:
!pip install scipy


In [5]:
import numpy as np
from stl import mesh

def calculate_tetrahedra_count(mesh_data):
    # Access the vertices and triangles from the mesh
    vertices = mesh_data.vectors.reshape(-1, 3)
    triangles = mesh_data.vectors

    # Initialize a set to store unique tetrahedra
    unique_tetrahedra = set()

    # Iterate over each triangle and form tetrahedra
    for triangle in triangles:
        tetrahedra = np.vstack([triangle, [0, 0, 0]])
        unique_tetrahedra.add(tuple(map(tuple, tetrahedra)))

    # Count the unique tetrahedra
    tetrahedra_count = len(unique_tetrahedra)

    return tetrahedra_count

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stliitm\\122472_314-208d_flat2_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
tetrahedra_count = calculate_tetrahedra_count(mesh_data)
print(f"Number of Tetrahedra: {tetrahedra_count}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\COE\\Desktop\\stliitm\\122472_314-208d_flat2_prt.stl'

In [ ]:
import numpy
import stl

def parse_stl(stl_file):

    # Load the STL file
    mesh_data = mesh.Mesh.from_file(stl_file)

    # Extract vertices and faces
    vertices = mesh_data.vectors.reshape((-1, 3))
    faces = np.arange(len(vertices)).reshape((-1, 3))

    return vertices, faces
face,vertices=parse_stl(r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl")
print(len(face))

In [ ]:
# import vtk

# # Load the STL file
# reader = vtk.vtkSTLReader()
# reader.SetFileName(r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl")
# reader.Update()

# # Get the number of cells (faces in an STL file)
# num_faces = reader.GetOutput().GetNumberOfCells()

# # Print the number of faces
# print("Number of faces in the STL file:", num_faces)

In [ ]:
import os
import numpy as np
from stl import mesh
import networkx as nx
from scipy.sparse.csgraph import connected_components

def calculate_connectivity_topology(mesh_data):
    """
    Calculate connectivity and topology features for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - num_connected_components: Number of connected components in the mesh
    """
    G = nx.Graph()
    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    adjacency_matrix = nx.adjacency_matrix(G)
    num_connected_components = connected_components(adjacency_matrix)[0]

    return num_connected_components

# Example usage
stl_file_path = r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
num_connected_components = calculate_connectivity_topology(mesh_data)
print(f"Number of Connected Components: {num_connected_components}")

In [ ]:
from sklearn.preprocessing import normalize

def calculate_vertex_normals_distribution(mesh_data):
    """
    Calculate the distribution of vertex normals for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - normal_distribution: Tuple containing mean and standard deviation of vertex normals
    """
    vertex_normals = normalize(mesh_data.normals)
    normal_distribution = (np.mean(vertex_normals, axis=0), np.std(vertex_normals, axis=0))

    return normal_distribution

# Example usage
stl_file_path =r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
normal_distribution = calculate_vertex_normals_distribution(mesh_data)
print(f"Normal Distribution Mean: {normal_distribution[0]}")
print(f"Normal Distribution Std: {normal_distribution[1]}")


In [ ]:
# print(nitisah)

In [ ]:
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np

def calculate_symmetry_planes(stl_file_path):
    # Read STL file using vtkSTLReader
    reader = vtk.vtkSTLReader()
    reader.SetFileName(stl_file_path)

    # Calculate normals
    normals = vtk.vtkPolyDataNormals()
    normals.SetInputConnection(reader.GetOutputPort())
    normals.ConsistencyOn()
    normals.SplittingOff()

    # Check if normals are successfully computed
    if normals.GetOutput().GetNumberOfPoints() == 0:
        print("Error: Unable to compute normals.")
        return None

    # Extract normals as numpy array
    normals_array = vtk_to_numpy(normals.GetOutput().GetPointData().GetNormals())

    # Check if normals_array is not empty
    if len(normals_array) == 0:
        print("Error: Empty normals array.")
        return None

    # Calculate the centroid of the mesh
    centroid = np.mean(normals_array, axis=0)

    # Use vtkTransformFilter to align the normals along the principal axes
    transform = vtk.vtkTransform()
    transform.Translate(-centroid[0], -centroid[1], -centroid[2])

    transformFilter = vtk.vtkTransformFilter()
    transformFilter.SetInputConnection(normals.GetOutputPort())
    transformFilter.SetTransform(transform)

    # Calculate the covariance matrix
    covarianceMatrix = vtk.vtkMatrix4x4()
    vtk.vtkMath.CovarianceMatrix(normals_array, covarianceMatrix)

    # Calculate the eigenvalues and eigenvectors
    eigenvalue, eigenvector = np.linalg.eigh(covarianceMatrix)

    # Use the eigenvector corresponding to the smallest eigenvalue as the symmetry plane normal
    symmetry_plane_normal = eigenvector[:, 0]

    return symmetry_plane_normal

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
symmetry_plane_normal = calculate_symmetry_planes(stl_file_path)

if symmetry_plane_normal is not None:
    print(f"Symmetry Plane Normal:\n{symmetry_plane_normal}")
else:
    print("Symmetry plane calculation failed.")


In [ ]:
!pip install vtk

In [ ]:
import vtk

# Load the STL file
reader = vtk.vtkSTLReader()
reader.SetFileName(r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl")
reader.Update()

# Calculate mean curvature
curve = vtk.vtkCurvatures()
curve.SetInputConnection(reader.GetOutputPort())
curve.SetCurvatureTypeToMean()  # Or Gaussian curvature if needed
curve.Update()

# Access curvature values
dataObject = curve.GetOutput()
curvature_array = dataObject.GetPointData().GetScalars()

print(curvature_array)

In [ ]:
import vtk

def calculate_mean_curvature(filename):
  """
  Calculates the mean curvature of each vertex in an STL file.

  Args:
      filename: The path to the STL file.

  Returns:
      A list of tuples, where each tuple contains the vertex coordinates and its mean curvature.
  """

  # Load the STL file
  reader = vtk.vtkSTLReader()
  reader.SetFileName(filename)
  reader.Update()

  # Calculate mean curvature
  curve = vtk.vtkCurvatures()
  curve.SetInputConnection(reader.GetOutputPort())
  curve.SetCurvatureTypeToMean()
  curve.Update()

  # Extract vertex coordinates and curvature values
  dataObject = curve.GetOutput()
  points = dataObject.GetPoints().GetData()
  curvature_array = dataObject.GetPointData().GetScalars()

  # Create a list of tuples
  vertex_curvatures = []
  for i in range(dataObject.GetNumberOfPoints()):
    point = points.GetTuple(i)
    curvature = curvature_array.GetValue(i)
    vertex_curvatures.append((point, curvature))

  return vertex_curvatures

# Example usage
filename = r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl"
vertex_curvatures = calculate_mean_curvature(filename)

for point, curvature in vertex_curvatures:
  print(f"Point: {point}, Curvature: {curvature}")

In [ ]:
def calculate_centroid(mesh_data):
    """
    Calculate the centroid for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - centroid: Centroid coordinates
    """
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)

    return centroid

# Example usage
stl_file_path = r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
centroid = calculate_centroid(mesh_data)
print(f"Centroid: {centroid}")


In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.mixture import GaussianMixture
import networkx as nx
from scipy.sparse.csgraph import connected_components
from sklearn.impute import SimpleImputer
def extract_holes_and_cavities(mesh_data):
    # Implementation for extracting holes and cavities
    # ...
    G = nx.Graph()

    # Add edges based on vertices of each facet
    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)  # Add individual vertices as nodes
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    # Euler characteristic: V - E + F = 2(1 - g - h)
    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)

    # Calculate the genus (g) and number of handles (h)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)
stl_file_path = r"C:\Users\COE\Desktop\stliitm\122472_314-208d_flat2_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
cavities = extract_holes_and_cavities(mesh_data)
print(f"Cavities: {cavities}")

Cavities: 67


In [ ]:
from stl_metadata import Surface,Facet

In [ ]:
import pickle

In [9]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import zipfile


zip_file_path = "/content/drive/MyDrive/datasets/stl_1000-20240321T094418Z-001.zip"


extract_dir = "/content/PDSVision_dataset/PDSVision_dataset"


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [7]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()

        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([surface_area]),
            np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    gmm.fit(X_scaled)

    with open('model.pkl', 'wb') as file:
        pickle.dump(gmm, file)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = r"/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
    output_folder = r"/content/drive/MyDrive/Datasets/evaluate_gmm_stl_1000"
    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 500
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")


FileNotFoundError: [Errno 2] No such file or directory: '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'

In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()

        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([surface_area]),
            np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    gmm.fit(X_scaled)

    with open('model.pkl', 'wb') as file:
        pickle.dump(gmm, file)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = r"C:\Users\COE\Desktop\stliitm"
    output_folder = r"C:\Users\COE\Desktop\stl_all-data_codes\all1000_gmm_centroid_centroid_sa_bbv"
    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 1000
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")


c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()

        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([surface_area]),
            np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    gmm.fit(X_scaled)

    with open('model.pkl', 'wb') as file:
        pickle.dump(gmm, file)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = r"C:\Users\COE\Desktop\stliitm"
    output_folder = r"C:\Users\COE\Desktop\stl_all-data_codes\all1500_gmm_centroid_centroid_sa_bbv"
    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 1500
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")


c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()
        x,y,z=surface.find_dims()
        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([x]),
            np.array([y]),
            np.array([z]),
            np.array([surface_area]),
            np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    gmm.fit(X_scaled)

    with open('model.pkl', 'wb') as file:
        pickle.dump(gmm, file)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = r"C:\Users\COE\Desktop\stliitm"
    output_folder = r"C:\Users\COE\Desktop\stl_all-data_codes\all2000_gmm_centroid_centroid_sa_bbv_lbh"
    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 2000
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")


c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


100% same: 146, 124, 118, 224, 295, 386, 424, 441, 475, 713, 743, 824, 844, 866, 995



slight : 127, 143, 757, 777, 812


absurd: 264, 282, 548, 669 , 894, 920, 945

In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()
        x,y,z=surface.find_dims()
        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([x]),
            np.array([y]),
            np.array([z]),
            np.array([surface_area]),
            np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    gmm.fit(X_scaled)

    with open('model.pkl', 'wb') as file:
        pickle.dump(gmm, file)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = r"C:\Users\COE\Desktop\stliitm"
    output_folder = r"C:\Users\COE\Desktop\stl_all-data_codes\all2500_gmm_centroid_centroid_sa_bbv_lbh"
    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 2500
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")


c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\COE\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
print(nitish_error)

NameError: name 'nitish_error' is not defined

In [ ]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
newX, new_stl = collect_features(r"C:\Users\COE\Desktop\test")
print(newX)
cluster_probs =  model.predict_proba(newX.reshape(1,-1))
predicted_cluster = cluster_probs.argmax()

print(predicted_cluster)

[[ 6.01813130e+00 -8.06757300e+00  9.35035000e-02  3.84087698e+02
   4.60551185e+01]]
484


In [ ]:
import os

def list_folders_and_files(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        lis = dirpath.split("/")
        print(lis[-1])
        for filename in filenames:
            print(f"{filename}")


In [ ]:

root_directory_path_1 ="C:\\Users\\COE\\Desktop\\GMM_cav_sa_1000"


list_folders_and_files(root_directory_path_1)


C:\Users\COE\Desktop\GMM_cav_sa_1000
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_0
130958_313-959d_flat1_prt.stl
134179_326-981d_prt.stl
134693_313-745d_flat1_prt.stl
135187_311-928d_prt.stl
135784_313-743d_flat1_prt.stl
143211_313-878d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_1
133786_307-309d_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_10
134997_313-986d_flat1_prt.stl
139414_313-893d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_100
134641_307-310h_asm.stl
134642_307-311h_asm.stl
138725_310-268d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_101
133125_311-490d_flat1_prt.stl
136895_311-492d_flat1_prt.stl
137843_311-488d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_102
123953_312-879d_flat1_prt.stl
131967_312-646d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_103
137556_317-004d_1stform_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000\Cluster_104
123540_310-605d_flat1_prt.stl
133581_310-474d_flat1_prt.stl
1363

In [ ]:

root_directory_path_1 ="C:\\Users\\COE\\Desktop\\GMM_cav_sa_1000_nocentroid"


list_folders_and_files(root_directory_path_1)


C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid\Cluster_0
129809_310-551d_flat1_prt.stl
130020_310-685d_flat1_prt.stl
130613_307-164d_flat1_prt.stl
130675_310-526d_flat1_prt.stl
131691_310-559du_prt.stl
132546_310-720du_prt.stl
132999_311-816d_flat1_prt.stl
133007_311-915d_flat_prt.stl
133460_311-551d_flat1_prt.stl
133888_313-912d_flat1_prt.stl
134699_311-916d_flat_prt.stl
135051_313-445d_flat1_prt.stl
135554_315-634d_flat1_prt.stl
136850_316-542d_flat1_prt.stl
137312_310-717du_prt.stl
137359_311-545d_flat1_prt.stl
137554_313-463d_flat2_prt.stl
137766_311-546d_flat1_prt.stl
138030_313-446d_flat1_prt.stl
138818_312-722d_flat1_prt.stl
140765_311-777du_prt.stl
140990_312-699d_flat1_prt.stl
141160_311-837d_flat1_prt.stl
141422_316-547d_flat_prt.stl
141516_314-226d_flat1_prt.stl
141633_311-920d_flat_prt.stl
141863_312-602d_flat1_prt.stl
142017_312-366d_flat1_prt.stl
142157_312-615d_flat1_prt.stl
143095_316-024d_flat1_prt.stl
143334_311-158d_fl

In [ ]:

root_directory_path_1 ="C:\\Users\\COE\\Desktop\\GMM_cav_sa_1000_nocentroid_nobbv"


list_folders_and_files(root_directory_path_1)


C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_0
131731_310-544d_flat1_prt.stl
131764_310-660d_flat1_prt.stl
138185_313-731d_prt.stl
138187_313-731d_flat1_prt.stl
138188_313-729d_flat1_prt.stl
138189_313-730d_flat1_prt.stl
140006_313-837d_flat1_prt.stl
141309_315-995d_press_brake_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_1
136787_313-634d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_10
142807_308-226d_flat1_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_100
130334_311-930h_asm.stl
130345_336-011h_asm.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_101
134885_312-823d_flat1_prt.stl
137556_317-004d_1stform_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_102
123653_310-610d_flat1_prt.stl
136072_311-746d_prt.stl
136073_311-747d_prt.stl
C:\Users\COE\Desktop\GMM_cav_sa_1000_nocentroid_nobbv\Cluster_103
1